# What is in the Trajectory JSONs?

- vocab

- splits (train, valid_seen, valid_unseen)

- batch (a bunch of examples)

- ex (a unit of task data)


https://github.com/Chucooleg/alfred/tree/explore_code/data

In [1]:
import os
os.environ['ALFRED_ROOT'] = '/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module

In [11]:
class args:
    pass

# settings
args.seed = 123
args.data = 'data/json_feat_2.1.0'
args.splits = 'data/splits/oct21.json'
args.preprocess = False #!
args.pp_folder = 'pp'
args.save_every_epoch = False #!
args.model = 'seq2seq_im'
args.gpu = True
args.dout = 'exp/model:seq2seq_im'
args.resume = False #!

# vocab objects

In [24]:
vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))

In [25]:
vocab

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}

In [27]:
vocab['action_low'].__dict__.keys()

dict_keys(['_index2word', '_word2index', 'counts'])

In [26]:
vocab['word'].__dict__.keys()

dict_keys(['_index2word', '_word2index', 'counts'])

In [28]:
[vocab['action_low'].index2word(i) for i in range(15)]

['<<pad>>',
 '<<seg>>',
 '<<stop>>',
 'LookDown_15',
 'MoveAhead_25',
 'RotateLeft_90',
 'PickupObject',
 'SliceObject',
 'LookUp_15',
 'RotateRight_90',
 'OpenObject',
 'PutObject',
 'CloseObject',
 'ToggleObjectOn',
 'ToggleObjectOff']

In [39]:
[vocab['action_high'].index2word(i) for i in range(93) if '<<' in vocab['word'].index2word(i)]

['<<pad>>', '<<seg>>', '<<stop>>', 'spatula']

In [41]:
# only top 20/93 words
[vocab['action_high'].index2word(i) for i in range(93)][:20]

['<<pad>>',
 '<<seg>>',
 '<<stop>>',
 'GotoLocation',
 'diningtable',
 'PickupObject',
 'knife',
 'SliceObject',
 'lettuce',
 'fridge',
 'PutObject',
 'CoolObject',
 'NoOp',
 'countertop',
 'candle',
 'cabinet',
 'toilet',
 'egg',
 'microwave',
 'HeatObject']

In [33]:
[vocab['word'].index2word(i) for i in range(2360) if '<' in vocab['word'].index2word(i)]

['<<pad>>', '<<seg>>', '<<goal>>', '<<stop>>']

In [52]:
# only top 20/2360 words
[vocab['word'].index2word(i) for i in range(2360)][:20]

['<<pad>>',
 '<<seg>>',
 '<<goal>>',
 'retrieve',
 'the',
 'sponge',
 'from',
 'kitchen',
 'island',
 ',',
 'place',
 'in',
 'fry',
 'pan',
 'put',
 'on',
 '.',
 'move',
 'to',
 'left']

# task data paths

In [50]:
# light data
!ls -l /home/hoyeung//alfred/data/json_2.1.0/train/pick_two_obj_and_place-WineBottle-None-GarbageCan-1/trial_T20190906_200307_446636/

total 208
-rw-r--r-- 1 538147517 500000513 212533 Apr  3 03:56 traj_data.json


In [49]:
# recommended data
!ls -l /home/hoyeung//alfred/data/json_feat_2.1.0/train/pick_two_obj_and_place-WineBottle-None-GarbageCan-1/trial_T20190906_200307_446636/

total 0
-rwxrwxrwx 1 root root 44556637 Apr  3 09:53 feat_conv.pt
-rwxrwxrwx 1 root root   212533 Apr  3 09:53 traj_data.json


In [51]:
# full data
!ls -l /home/hoyeung//alfred/data/full_2.1.0/train/pick_two_obj_and_place-WineBottle-None-GarbageCan-1/trial_T20190906_200307_446636/

total 20
-rwxrwxrwx 1 root root 44556637 Apr  3 21:22 feat_conv.pt
-rwxrwxrwx 1 root root   228450 Apr  3 21:22 problem_0.pddl
drwxrwxrwx 2 root root    20480 Apr  5 16:49 raw_images
-rwxrwxrwx 1 root root   212533 Apr  3 21:22 traj_data.json
-rwxrwxrwx 1 root root  1911057 Apr  3 21:22 video.mp4


# splits


- Load here
https://github.com/Chucooleg/alfred/blob/6d2a6d9b210ea2ab57a3d6c6b2810f796e9ad2d1/models/train/train_seq2seq.py#L80

In [12]:
# load train/valid/tests splits
with open(args.splits) as f:
    splits = json.load(f)
    pprint.pprint({k: len(v) for k, v in splits.items()})

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 21023,
 'valid_seen': 820,
 'valid_unseen': 821}


In [15]:
type(splits['train'])

list

In [16]:
splits['train'][0]

{'repeat_idx': 0,
 'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648'}

In [17]:
splits['valid_seen'][0]

{'repeat_idx': 0,
 'task': 'pick_heat_then_place_in_recep-PotatoSliced-None-SinkBasin-13/trial_T20190909_115736_122556'}

In [18]:
splits['valid_unseen'][0]

{'repeat_idx': 0,
 'task': 'look_at_obj_in_light-CellPhone-None-FloorLamp-219/trial_T20190908_044113_026049'}

In [19]:
splits['tests_seen'][0]

{'repeat_idx': 0, 'task': 'trial_T20190909_042500_949430'}

In [21]:
splits['tests_unseen'][0]

{'repeat_idx': 0, 'task': 'trial_T20190906_190741_530882'}

# Batch

- Loaded here https://github.com/Chucooleg/alfred/blob/6d2a6d9b210ea2ab57a3d6c6b2810f796e9ad2d1/models/model/seq2seq.py#L277

In [43]:
args.data

'data/json_feat_2.1.0'

In [19]:
def load_task_json(task):
    '''
    load preprocessed json from disk
    '''
    json_path = os.path.join(args.data, task['task'], 'pp', 'ann_%d.json' % task['repeat_idx'])
    print(json_path)
    with open(json_path) as f:
        data = json.load(f)
    return data

train = splits['train']
tasks = train[:100]
print("TASK")
print(tasks[0])
print(tasks[1])
print(tasks[2])
print(tasks[3])
print(tasks[4])
print(tasks[5])
print('-----------------------------------')
print("BATCH json_path")
batch = [load_task_json(task) for task in tasks]

TASK
{'repeat_idx': 0, 'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648'}
{'repeat_idx': 1, 'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648'}
{'repeat_idx': 2, 'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648'}
{'repeat_idx': 0, 'task': 'pick_two_obj_and_place-Candle-None-Cabinet-414/trial_T20190908_190650_163902'}
{'repeat_idx': 1, 'task': 'pick_two_obj_and_place-Candle-None-Cabinet-414/trial_T20190908_190650_163902'}
{'repeat_idx': 2, 'task': 'pick_two_obj_and_place-Candle-None-Cabinet-414/trial_T20190908_190650_163902'}
-----------------------------------
BATCH json_path
/root/data_alfred/json_feat_2.1.0/pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/ann_0.json
/root/data_alfred/json_feat_2.1.0/pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_

/root/data_alfred/json_feat_2.1.0/pick_and_place_with_movable_recep-Fork-Mug-SinkBasin-18/trial_T20190909_043700_281076/pp/ann_0.json
/root/data_alfred/json_feat_2.1.0/pick_and_place_with_movable_recep-Fork-Mug-SinkBasin-18/trial_T20190909_043700_281076/pp/ann_1.json
/root/data_alfred/json_feat_2.1.0/pick_and_place_with_movable_recep-Fork-Mug-SinkBasin-18/trial_T20190909_043700_281076/pp/ann_2.json
/root/data_alfred/json_feat_2.1.0/pick_heat_then_place_in_recep-Tomato-None-CounterTop-26/trial_T20190907_005525_499114/pp/ann_0.json
/root/data_alfred/json_feat_2.1.0/pick_heat_then_place_in_recep-Tomato-None-CounterTop-26/trial_T20190907_005525_499114/pp/ann_1.json
/root/data_alfred/json_feat_2.1.0/pick_heat_then_place_in_recep-Tomato-None-CounterTop-26/trial_T20190907_005525_499114/pp/ann_2.json
/root/data_alfred/json_feat_2.1.0/pick_and_place_with_movable_recep-Spatula-Pan-SinkBasin-4/trial_T20190907_160825_608912/pp/ann_0.json
/root/data_alfred/json_feat_2.1.0/pick_and_place_with_movabl

In [23]:
for i, task in enumerate(train):
    if 'mov' in task['task']:
        print(i, task['task'])

40 pick_and_place_with_movable_recep-ButterKnife-Cup-DiningTable-23/trial_T20190909_224509_434167
41 pick_and_place_with_movable_recep-ButterKnife-Cup-DiningTable-23/trial_T20190909_224509_434167
42 pick_and_place_with_movable_recep-ButterKnife-Cup-DiningTable-23/trial_T20190909_224509_434167
64 pick_and_place_with_movable_recep-Knife-Pot-DiningTable-24/trial_T20190907_141859_421899
65 pick_and_place_with_movable_recep-Knife-Pot-DiningTable-24/trial_T20190907_141859_421899
66 pick_and_place_with_movable_recep-Knife-Pot-DiningTable-24/trial_T20190907_141859_421899
67 pick_and_place_with_movable_recep-Knife-Pot-DiningTable-24/trial_T20190907_141859_421899
77 pick_and_place_with_movable_recep-Fork-Mug-SinkBasin-18/trial_T20190909_043700_281076
78 pick_and_place_with_movable_recep-Fork-Mug-SinkBasin-18/trial_T20190909_043700_281076
79 pick_and_place_with_movable_recep-Fork-Mug-SinkBasin-18/trial_T20190909_043700_281076
83 pick_and_place_with_movable_recep-Spatula-Pan-SinkBasin-4/trial_T201

6106 pick_and_place_with_movable_recep-Pen-Mug-Dresser-324/trial_T20190909_145723_528863
6107 pick_and_place_with_movable_recep-Pen-Mug-Dresser-324/trial_T20190909_145723_528863
6129 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-224/trial_T20190911_041655_093014
6130 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-224/trial_T20190911_041655_093014
6131 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-224/trial_T20190911_041655_093014
6165 pick_and_place_with_movable_recep-ButterKnife-Plate-DiningTable-19/trial_T20190919_051826_332624
6166 pick_and_place_with_movable_recep-ButterKnife-Plate-DiningTable-19/trial_T20190919_051826_332624
6167 pick_and_place_with_movable_recep-ButterKnife-Plate-DiningTable-19/trial_T20190919_051826_332624
6180 pick_and_place_with_movable_recep-LettuceSliced-Pan-Fridge-25/trial_T20190908_073534_320526
6181 pick_and_place_with_movable_recep-LettuceSliced-Pan-Fridge-25/trial_T20190908_073534_320526
6182 pick_and_place_wi

12442 pick_and_place_with_movable_recep-RemoteControl-Box-Desk-204/trial_T20190907_145909_032296
12446 pick_and_place_with_movable_recep-Spoon-Pot-DiningTable-21/trial_T20190918_203540_394703
12447 pick_and_place_with_movable_recep-Spoon-Pot-DiningTable-21/trial_T20190918_203540_394703
12448 pick_and_place_with_movable_recep-Spoon-Pot-DiningTable-21/trial_T20190918_203540_394703
12452 pick_and_place_with_movable_recep-KeyChain-Plate-Shelf-214/trial_T20190908_221228_624762
12453 pick_and_place_with_movable_recep-KeyChain-Plate-Shelf-214/trial_T20190908_221228_624762
12454 pick_and_place_with_movable_recep-KeyChain-Plate-Shelf-214/trial_T20190908_221228_624762
12475 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-202/trial_T20190908_071526_644521
12476 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-202/trial_T20190908_071526_644521
12477 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-202/trial_T20190908_071526_644521
12490 pick_and_place_with_mova

18607 pick_and_place_with_movable_recep-Spatula-Pot-CounterTop-26/trial_T20190907_215117_439685
18608 pick_and_place_with_movable_recep-Spatula-Pot-CounterTop-26/trial_T20190907_215117_439685
18609 pick_and_place_with_movable_recep-Spatula-Pot-CounterTop-26/trial_T20190907_215117_439685
18622 pick_and_place_with_movable_recep-Knife-Mug-SideTable-3/trial_T20190906_182114_677622
18623 pick_and_place_with_movable_recep-Knife-Mug-SideTable-3/trial_T20190906_182114_677622
18624 pick_and_place_with_movable_recep-Knife-Mug-SideTable-3/trial_T20190906_182114_677622
18625 pick_and_place_with_movable_recep-Knife-Mug-SideTable-3/trial_T20190906_182114_677622
18626 pick_and_place_with_movable_recep-Knife-Mug-SideTable-3/trial_T20190906_182114_677622
18645 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-224/trial_T20190911_044205_765229
18646 pick_and_place_with_movable_recep-RemoteControl-Box-ArmChair-224/trial_T20190911_044205_765229
18647 pick_and_place_with_movable_recep-RemoteCont

# ex / task data

- One item in Batch

In [24]:
ex = batch[40]

In [58]:
print(ex.keys())

dict_keys(['ann', 'images', 'num', 'pddl_params', 'plan', 'root', 'scene', 'split', 'task_id', 'task_type', 'turk_annotations'])


In [65]:
# annotation
print(ex['ann'].keys())
print('-------------------------')
print(ex['ann']['goal'])
print('-------------------------')
print(type(ex['ann']['instr']))
print(type(ex['ann']['instr'][0]))
pprint.pprint(ex['ann']['instr'])

dict_keys(['goal', 'instr', 'repeat_idx'])
-------------------------
[' put ', ' a ', ' slice ', ' of ', ' vegetable ', ' on ', ' a ', ' counter ', '. ', '<<goal>>']
-------------------------
<class 'list'>
<class 'list'>
[[' walk ',
  ' straight ',
  ', ',
  ' turn ',
  ' left ',
  ' to ',
  ' the ',
  ' counter ',
  '. '],
 [' pick ', ' up ', ' the ', ' knife ', ' on ', ' the ', ' table ', '. '],
 [' slice ',
  ' the ',
  ' green ',
  ' vegetable ',
  ' on ',
  ' the ',
  ' table ',
  '. '],
 [' walk ',
  ' straight ',
  ', ',
  ' turn ',
  ' right ',
  ' to ',
  ' the ',
  ' fridge ',
  '. '],
 [' open ',
  ' the ',
  ' fridge ',
  ', ',
  ' put ',
  ' the ',
  ' knife ',
  ' inside ',
  ' the ',
  ' fridge ',
  '. ',
  ' close ',
  ' the ',
  ' fridge '],
 [' turn ',
  ' right ',
  ', ',
  ' walk ',
  ' straight ',
  ', ',
  ' turn ',
  ' left ',
  ' to ',
  ' the ',
  ' counter ',
  '. '],
 [' take ', ' a ', ' slice ', ' of ', ' the ', ' vegetable ', '. '],
 [' turn ',
  ' left ',

In [83]:
# images
print(type(ex['images']))
print('-------------------------')
print(type(ex['images'][0]))
print(ex['images'][0].keys())
print('-------------------------')
print('length of images', len(ex['images']))
print('--------------------------------------------')
print('high_idx : ', ex['images'][0]['high_idx'])
print('-------------------------')
print('low_idx  : ', ex['images'][0]['low_idx'])
print('-------------------------')
print(ex['images'][0]['image_name'])

print('--------------------------------------------')
print('high_idx : ', ex['images'][1]['high_idx'])
print('-------------------------')
print('low_idx  : ', ex['images'][1]['low_idx'])
print('-------------------------')
print(ex['images'][1]['image_name'])

print('--------------------------------------------')
print('high_idx : ', ex['images'][121]['high_idx'])
print('-------------------------')
print('low_idx  : ', ex['images'][121]['low_idx'])
print('-------------------------')
print(ex['images'][121]['image_name'])

print('--------------------------------------------')
print('high_idx : ', ex['images'][242]['high_idx'])
print('-------------------------')
print('low_idx  : ', ex['images'][242]['low_idx'])
print('-------------------------')
print(ex['images'][242]['image_name'])

print('--------------------------------------------')
print('high_idx : ', ex['images'][-1]['high_idx'])
print('-------------------------')
print('low_idx  : ', ex['images'][-1]['low_idx'])
print('-------------------------')
print(ex['images'][-1]['image_name'])


<class 'list'>
-------------------------
<class 'dict'>
dict_keys(['high_idx', 'image_name', 'low_idx'])
-------------------------
length of images 455
--------------------------------------------
high_idx :  0
-------------------------
low_idx  :  0
-------------------------
000000000.png
--------------------------------------------
high_idx :  0
-------------------------
low_idx  :  0
-------------------------
000000001.png
--------------------------------------------
high_idx :  3
-------------------------
low_idx  :  21
-------------------------
000000121.png
--------------------------------------------
high_idx :  7
-------------------------
low_idx  :  37
-------------------------
000000242.png
--------------------------------------------
high_idx :  10
-------------------------
low_idx  :  66
-------------------------
000000454.png


In [89]:
# num -- integer repr of actions and language
print(ex['num'].keys())
print('-------------------------')
print("action_high")
pprint.pprint(ex['num']['action_high'])
print('-------------------------')
print("action_low")
pprint.pprint(ex['num']['action_low'])
print('-------------------------')

dict_keys(['action_high', 'action_low', 'lang_goal', 'lang_instr', 'low_to_high_idx'])
-------------------------
action_high
[{'action': 3, 'action_high_args': [4], 'high_idx': 0},
 {'action': 5, 'action_high_args': [6], 'high_idx': 1},
 {'action': 7, 'action_high_args': [8], 'high_idx': 2},
 {'action': 3, 'action_high_args': [9], 'high_idx': 3},
 {'action': 10, 'action_high_args': [6, 9], 'high_idx': 4},
 {'action': 3, 'action_high_args': [4], 'high_idx': 5},
 {'action': 5, 'action_high_args': [8], 'high_idx': 6},
 {'action': 3, 'action_high_args': [9], 'high_idx': 7},
 {'action': 11, 'action_high_args': [8], 'high_idx': 8},
 {'action': 3, 'action_high_args': [4], 'high_idx': 9},
 {'action': 10, 'action_high_args': [8, 4], 'high_idx': 10},
 {'action': 12, 'action_high_args': [], 'high_idx': 11}]
-------------------------
action_low
[[{'action': 3,
   'action_high_args': {},
   'centroid': [-1, -1],
   'high_idx': 0,
   'mask': None,
   'valid_interact': 0},
  {'action': 4,
   'action_

   'action_high_args': {'bbox': [132, 98, 138, 132],
                        'mask': [[29233, 4],
                                 [29533, 6],
                                 [29833, 6],
                                 [30133, 6],
                                 [30433, 6],
                                 [30733, 6],
                                 [31033, 6],
                                 [31333, 6],
                                 [31633, 6],
                                 [31933, 6],
                                 [32233, 6],
                                 [32532, 7],
                                 [32832, 7],
                                 [33132, 7],
                                 [33432, 7],
                                 [33732, 7],
                                 [34032, 7],
                                 [34332, 7],
                                 [34632, 7],
                                 [34932, 7],
                                 [35232, 7],
  

                                 [63535, 65],
                                 [63835, 65],
                                 [64135, 65],
                                 [64436, 64],
                                 [64736, 64],
                                 [65036, 64],
                                 [65336, 64],
                                 [65636, 64],
                                 [65936, 64],
                                 [66237, 63],
                                 [66537, 63],
                                 [66837, 63],
                                 [67137, 63],
                                 [67437, 63],
                                 [67737, 63],
                                 [68038, 62],
                                 [68338, 62],
                                 [68638, 62],
                                 [68938, 62],
                                 [69238, 62],
                                 [69538, 62],
                                 [

In [90]:
print("lang_goal")
pprint.pprint(ex['num']['lang_goal'])
print('-------------------------')
print("lang_instr")
pprint.pprint(ex['num']['lang_instr'])
print('-------------------------')
print("low_to_high_idx")
pprint.pprint(ex['num']['low_to_high_idx'])

lang_goal
[14, 35, 55, 21, 477, 15, 35, 36, 16, 2]
-------------------------
lang_instr
[[47, 257, 9, 24, 19, 18, 4, 36, 16],
 [22, 23, 4, 63, 15, 4, 60, 16],
 [55, 4, 292, 477, 15, 4, 60, 16],
 [47, 257, 9, 24, 38, 18, 4, 85, 16],
 [82, 4, 85, 9, 14, 4, 63, 68, 4, 85, 16, 84, 4, 85],
 [24, 38, 9, 47, 257, 9, 24, 19, 18, 4, 36, 16],
 [69, 35, 55, 21, 4, 477, 16],
 [24, 19, 9, 47, 257, 9, 24, 38, 18, 4, 85, 16],
 [82,
  4,
  85,
  9,
  14,
  4,
  55,
  73,
  18,
  4,
  63,
  16,
  84,
  4,
  85,
  16,
  82,
  4,
  85,
  9,
  69,
  4,
  55,
  9,
  84,
  4,
  85,
  16],
 [24, 38, 9, 47, 257, 9, 24, 38, 9, 24, 38, 9, 24, 38, 18, 4, 36, 16],
 [14, 4, 55, 15, 4, 36, 16],
 [34]]
-------------------------
low_to_high_idx
[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 11]


In [91]:
# pddl_params
print(ex['pddl_params'].keys())
print('-------------------------')

dict_keys(['mrecep_target', 'object_sliced', 'object_target', 'parent_target', 'toggle_target'])
-------------------------


In [25]:
ex['plan']['high_pddl']

[{'discrete_action': {'action': 'GotoLocation', 'args': ['diningtable']},
  'high_idx': 0,
  'planner_action': {'action': 'GotoLocation', 'location': 'loc|-7|-6|3|45'}},
 {'discrete_action': {'action': 'PickupObject', 'args': ['butterknife']},
  'high_idx': 1,
  'planner_action': {'action': 'PickupObject',
   'coordinateObjectId': ['ButterKnife',
    [-10.2110424,
     -10.2110424,
     -5.69593908,
     -5.69593908,
     3.694068672,
     3.694068672]],
   'coordinateReceptacleObjectId': ['DiningTable',
    [-9.73199272, -9.73199272, -6.78581428, -6.78581428, 3.5828, 3.5828]],
   'forceVisible': True,
   'objectId': 'ButterKnife|-02.55|+00.92|-01.42'}},
 {'discrete_action': {'action': 'GotoLocation', 'args': ['cup']},
  'high_idx': 2,
  'planner_action': {'action': 'GotoLocation', 'location': 'loc|-4|-3|1|45'}},
 {'discrete_action': {'action': 'PutObject', 'args': ['butterknife', 'cup']},
  'high_idx': 3,
  'planner_action': {'action': 'PutObject',
   'coordinateObjectId': ['ButterKni

In [130]:
# plan
print(ex['plan'].keys())
print('-------------------------')
pprint.pprint([a['discrete_action']['action'] for a in ex['plan']['low_actions']])
print('-------------------------')
pprint.pprint([a['discrete_action']['action'] for a in ex['plan']['high_pddl']])

dict_keys(['high_pddl', 'low_actions'])
-------------------------
['LookDown_15',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'LookDown_15',
 'PickupObject',
 'SliceObject',
 'LookUp_15',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateRight_90',
 'LookDown_15',
 'OpenObject',
 'PutObject',
 'CloseObject',
 'LookUp_15',
 'RotateRight_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateLeft_90',
 'LookDown_15',
 'PickupObject',
 'LookUp_15',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateRight_90',
 'LookDown_15',
 'OpenObject',
 'PutObject',
 'CloseObject',
 'OpenObject',
 'PickupObject',
 'CloseObject',
 'LookUp_15',
 'RotateRight_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'M

In [96]:
# root
print(ex['root'])
print('-------------------------')

data/json_feat_2.1.0/pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648
-------------------------


In [102]:
# root
print(ex['scene'].keys())
print('-------------------------')
print(ex['scene']['dirty_and_empty'])
print('-------------------------')
print(ex['scene']['floor_plan'])
print('-------------------------')
print(ex['scene']['init_action'])
print('-------------------------')
print(ex['scene']['object_poses'])

dict_keys(['dirty_and_empty', 'floor_plan', 'init_action', 'object_poses', 'object_toggles', 'random_seed', 'scene_num'])
-------------------------
False
-------------------------
FloorPlan17
-------------------------
{'action': 'TeleportFull', 'horizon': 30, 'rotateOnTeleport': True, 'rotation': 0, 'x': 0.25, 'y': 0.908999562, 'z': 1.5}
-------------------------
[{'objectName': 'Potato_edec52fd', 'position': {'x': -0.0824483, 'y': 0.803487062, 'z': -0.533726156}, 'rotation': {'x': 1.40334191e-14, 'y': 0.0, 'z': 0.0}}, {'objectName': 'DishSponge_4b57442b', 'position': {'x': -0.503703535, 'y': 0.911664248, 'z': -0.430464923}, 'rotation': {'x': 0.0, 'y': 0.0, 'z': 0.0}}, {'objectName': 'ButterKnife_432df619', 'position': {'x': 1.36722, 'y': 0.8863421, 'z': 2.692625}, 'rotation': {'x': 0.0, 'y': 0.0, 'z': 0.0}}, {'objectName': 'ButterKnife_432df619', 'position': {'x': 1.128434, 'y': 0.9115421, 'z': -0.144580841}, 'rotation': {'x': 0.0, 'y': 0.0, 'z': 0.0}}, {'objectName': 'Glassbottle_601

In [106]:
print('-------------------------')
print(ex['scene']['object_toggles'])
print('-------------------------')
print(ex['scene']['random_seed'])
print('-------------------------')
print(ex['scene']['scene_num'])

-------------------------
[]
-------------------------
3517829618
-------------------------
17


In [108]:
# split
print(ex['split'])
print('-------------------------')

train
-------------------------


In [109]:
# task_id
print(ex['task_id'])
print('-------------------------')

trial_T20190909_070538_437648
-------------------------


In [110]:
# task_type
print(ex['task_type'])
print('-------------------------')

pick_cool_then_place_in_recep
-------------------------


In [121]:
# turk_annotations
# -- weird that each task data is specific to an annotation in ex['ann'] but include all 3 in ex['turk_annotations']['anns']
print(ex['turk_annotations'].keys())
print('-------------------------')
print(type(ex['turk_annotations']['anns']))
print(len(ex['turk_annotations']['anns']))
print('-------------------------')
pprint.pprint(ex['turk_annotations']['anns'][0])
print('-------------------------')
pprint.pprint(ex['turk_annotations']['anns'][1])
print('-------------------------')
pprint.pprint(ex['turk_annotations']['anns'][2])

dict_keys(['anns'])
-------------------------
<class 'list'>
3
-------------------------
{'assignment_id': 'ARB80JPV0HQ5O_34V1S5K3GVSAEXO2MYUEMPSBO6G69Y',
 'high_descs': ['Walk straight, turn left to the counter.',
                'Pick up the knife on the table.',
                'Slice the green vegetable on the table.',
                'Walk straight, turn right to the fridge.',
                'Open the fridge, put the knife inside the fridge. Close the '
                'fridge',
                'Turn right, walk straight, turn left to the counter.',
                'Take a slice of the vegetable.',
                'Turn left, walk straight, turn right to the fridge.',
                'Open the fridge, put the slice next to the knife. Close the '
                'fridge. Open the fridge, take the slice, close the fridge.',
                'Turn right, walk straight, turn right, turn right, turn right '
                'to the counter.',
                'Put the slice on the counte